In [36]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [37]:
X_train_path= os.path.join("..", "..", "data","enriched_input", "X_train.csv")
X_train = pd.read_csv(X_train_path, delimiter=',')

X_test_path= os.path.join("..", "..", "data","enriched_input", "X_test.csv")
X_test = pd.read_csv(X_test_path, delimiter=',')
delivery_start = X_test["DELIVERY_START"]

y_train_path= os.path.join("..", "..", "data","enriched_input", "y_train.csv")
y_train = pd.read_csv(y_train_path, delimiter=',')

In [38]:
X_train.set_index("DELIVERY_START", inplace=True)
X_test.set_index("DELIVERY_START", inplace=True)
y_train.set_index("DELIVERY_START", inplace=True)
X_train.index = pd.to_datetime(X_train.index, utc=True)
X_test.index = pd.to_datetime(X_test.index, utc=True)
y_train.index = pd.to_datetime(y_train.index, utc=True)

In [39]:
X_train = X_train.drop(columns=['year'])
X_test = X_test.drop(columns=['year'])

In [40]:
X_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,cold_rate,av_consuption_hour,hour,dayofweek,quarter,month,dayofyear
DELIVERY_START,,,,,,,,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887,-1.112973,-0.775054,-1.519140,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887,-1.112973,-1.010691,-1.374581,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887,-1.112973,-1.565537,-1.230023,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887,-1.112973,-1.991068,-1.085464,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887,-1.112973,-2.026920,-0.940906,1.002602,-1.045267,-1.287117,-1.406310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128,-0.492362,-0.085946,0.793798,-0.489343,-1.045267,-0.734099,-0.618926
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000,-0.492362,0.271836,0.938356,-0.489343,-1.045267,-0.734099,-0.618926
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887,-0.492362,0.969350,1.082915,-0.489343,-1.045267,-0.734099,-0.618926


In [41]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

Classifier

In [42]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train_direction)
y_pred_direction = clf.predict(X_test)

Regressor

In [43]:
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train_magnitude)
y_pred_magnitude = reg.predict(X_test)

In [44]:
y_pred = reconstruct_y(y_pred_direction, y_pred_magnitude)

In [45]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

#y_pred.to_csv("../../data/output_y_test/y_result_double_rd_forest2.csv", index = False)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [47]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])
y_test_direction, y_test_magnitude = decompose_y(y_test['spot_id_delta'])

In [48]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train_direction)
y_pred_direction = clf.predict(X_test)
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train_magnitude)
y_pred_magnitude = reg.predict(X_test)

In [49]:
y_pred = reconstruct_y(y_pred_direction, y_pred_magnitude)

In [50]:
rmse_voting = np.sqrt(mean_squared_error(y_test, y_pred))

In [52]:
rmse_voting

19.229675763791843